<a href="https://colab.research.google.com/github/swimskykim00/SW-project/blob/main/%EA%B0%9C%EC%9D%B8%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B82(Mvtec).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


!xz -d /content/drive/MyDrive/SW사업/carpet.tar.xz

In [3]:
!tar xvf /content/drive/MyDrive/SW사업/carpet.tar

carpet/
carpet/ground_truth/
carpet/ground_truth/color/
carpet/ground_truth/color/016_mask.png
carpet/ground_truth/color/017_mask.png
carpet/ground_truth/color/018_mask.png
carpet/ground_truth/color/000_mask.png
carpet/ground_truth/color/001_mask.png
carpet/ground_truth/color/002_mask.png
carpet/ground_truth/color/003_mask.png
carpet/ground_truth/color/004_mask.png
carpet/ground_truth/color/005_mask.png
carpet/ground_truth/color/006_mask.png
carpet/ground_truth/color/007_mask.png
carpet/ground_truth/color/008_mask.png
carpet/ground_truth/color/009_mask.png
carpet/ground_truth/color/010_mask.png
carpet/ground_truth/color/011_mask.png
carpet/ground_truth/color/012_mask.png
carpet/ground_truth/color/013_mask.png
carpet/ground_truth/color/014_mask.png
carpet/ground_truth/color/015_mask.png
carpet/ground_truth/cut/
carpet/ground_truth/cut/000_mask.png
carpet/ground_truth/cut/001_mask.png
carpet/ground_truth/cut/002_mask.png
carpet/ground_truth/cut/003_mask.png
carpet/ground_truth/cut/004_ma

In [4]:
import os, glob
import tensorflow as tf
from tensorflow.keras import layers, models
# 이거 폴더 안의 모든 거에 대한 경로가 다 담겨져있음 리스트형태로

# ground_truth 파일 경로
# data_dir_ground_color = glob.glob("/content/carpet/ground_truth/color/*")
# data_dir_ground_cut = glob.glob("/content/carpet/ground_truth/cut/*")
# data_dir_ground_hole = glob.glob("/content/carpet/ground_truth/hole/*")
# data_dir_ground_metal_contam = glob.glob("/content/carpet/ground_truth/metal_contamination/*")
# data_dir_ground_thread = glob.glob("/content/carpet/ground_truth/thread/*")

# test 파일 경로
data_dir_test_color = glob.glob("/content/carpet/test/color/*")
data_dir_test_cut = glob.glob("/content/carpet/test/cut/*")
data_dir_test_good = glob.glob("/content/carpet/test/good/*")
data_dir_test_hole = glob.glob("/content/carpet/test/hole/*")
data_dir_test_metal_contam = glob.glob("/content/carpet/test/metal_contamination/*")
data_dir_test_thread = glob.glob("/content/carpet/test/thread/*")

# train 파일 경로
data_dir_train_good = glob.glob("/content/carpet/train/good/*")

In [ ]:
# from PIL import Image
# img = Image.open(data_dir_test_color[12])
# img
# import numpy as np
# np_image = np.array(img)
# np_image.shape

(1024, 1024, 3)

In [5]:
from PIL import Image
import numpy as np

# Combine all the file paths in a list
file_paths_train = data_dir_train_good
list_file_path_test = [data_dir_test_color, data_dir_test_cut, data_dir_test_good,data_dir_test_hole,data_dir_test_metal_contam,data_dir_test_thread ]

train_image_list = []
test_color_image_list = []
test_cut_image_list = []
test_good_image_list =[]
test_hole_image_list = []
test_metal_contam_image_list = []
test_thread_image_list = []

all_list_for_test = [test_color_image_list, test_cut_image_list, test_good_image_list, test_hole_image_list, test_metal_contam_image_list, test_thread_image_list]

for image in data_dir_train_good:
    # Open the image using PIL
    img = Image.open(image)

    # Resize the image to (512, 512)
    img = img.resize((512, 512))

    # Convert the image to a NumPy array and scale pixel values to range 0-1
    np_image = np.array(img) / 255.0

    train_image_list.append(np_image)

# Iterate over each class-specific list and file path list
for class_list, file_paths in zip(all_list_for_test, list_file_path_test):
    # Iterate over each file path in the current class
    for file_path in file_paths:
        # Open the image using PIL
        img = Image.open(file_path)
        # Resize the image to (512, 512)
        img = img.resize((512, 512))
        # Convert the image to a NumPy array
        np_image = np.array(img) / 255.0

        # Append the NumPy array to the current class-specific list
        class_list.append(np_image)

In [6]:
import pandas as pd

# Create a list of dataframes and corresponding target labels
dataframes = [test_color_image_list, test_cut_image_list, test_hole_image_list,
              test_metal_contam_image_list, test_thread_image_list, test_good_image_list]

target_labels = [1, 1, 1, 1, 1, 0]

# Create dataframes and concatenate them
dfs = [pd.DataFrame({'image': images, 'target': [target] * len(images)}) for images, target in zip(dataframes, target_labels)]
df_test = pd.concat(dfs, ignore_index=True)

# Print the combined dataframe
print(df_test)

# Create a DataFrame for the train data
df_train = pd.DataFrame({'image': train_image_list, 'target': [0] * len(train_image_list)})

print(df_train)



                                                 image  target
0    [[[0.1607843137254902, 0.1568627450980392, 0.1...       1
1    [[[0.2823529411764706, 0.2549019607843137, 0.2...       1
2    [[[0.12549019607843137, 0.13333333333333333, 0...       1
3    [[[0.16470588235294117, 0.15294117647058825, 0...       1
4    [[[0.3137254901960784, 0.2784313725490196, 0.2...       1
..                                                 ...     ...
112  [[[0.2980392156862745, 0.2901960784313726, 0.2...       0
113  [[[0.21176470588235294, 0.1803921568627451, 0....       0
114  [[[0.25098039215686274, 0.24313725490196078, 0...       0
115  [[[0.3058823529411765, 0.29411764705882354, 0....       0
116  [[[0.21568627450980393, 0.1843137254901961, 0....       0

[117 rows x 2 columns]
                                                 image  target
0    [[[0.22745098039215686, 0.20784313725490197, 0...       0
1    [[[0.44313725490196076, 0.4549019607843137, 0....       0
2    [[[0.2549019607843137, 0.2

In [7]:
df1 = df_test.copy()
df2 = df_train.copy()

In [ ]:
# # Save the dataframe to a CSV file
# df.to_csv('/content/mvtec_carpet_final_df.csv', index=False)
# import shutil
# shutil.move('/content/mvtec_carpet_final_df.csv', '/content/drive/MyDrive/')

In [8]:
from sklearn.model_selection import train_test_split

X_train = df_train['image']
y_train = df_train['target']
X_test = df_test['image']
y_test = df_test['target']

# print(X_train)
# print(y_train)
# print(X_test)
# print(y_test)

print(type(X_train))
print(type(X_test))


# Split the test data into training  testing sets
X_train_classify, X_test_classify, y_train_classify, y_test_classify = train_test_split(X_test, y_test, test_size=0.2, stratify=y_test, random_state = 1024)


# change the series object into a tensor object so we can input it into a model
X_train = np.array(X_train.tolist())
X_test = np.array(X_test.tolist())
y_train = np.array(y_train.tolist())
y_test = np.array(y_test.tolist())

X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

X_train_classify = np.array(X_train_classify.tolist())
X_test_classify = np.array(X_test_classify.tolist())
y_train_classify = np.array(y_train_classify.tolist())
y_test_classify = np.array(y_test_classify.tolist())

X_train_classify = tf.convert_to_tensor(X_train_classify, dtype=tf.float32)
X_test_classify = tf.convert_to_tensor(X_test_classify, dtype=tf.float32)
y_train_classify = tf.convert_to_tensor(y_train_classify, dtype=tf.float32)
y_test_classify = tf.convert_to_tensor(y_test_classify, dtype=tf.float32)

type(X_train)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


tensorflow.python.framework.ops.EagerTensor

In [10]:
print(X_train.shape)
print(X_test.shape)


(280, 512, 512, 3)
(117, 512, 512, 3)


In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models

input_shape = (512, 512, 3)

# Encoder
encoder_input = tf.keras.Input(shape=input_shape)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoder_input)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)

# Bottleneck layer
encoded = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)

# Decoder
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)

# Dropout layer to prevent overfitting
x = layers.Dropout(0.2)(x)

decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Autoencoder model
autoencoder = models.Model(encoder_input, decoded)

# Compile the model
autoencoder.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Print the model summary
autoencoder.summary()




Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 512, 512, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 512, 512, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 256, 256, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 128, 128, 64)     0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 128, 128)     73856 

In [10]:
# Assuming X_train contains your training data
autoencoder.fit(X_train, X_train, epochs=100, batch_size=32)


Epoch 1/100
9/9 [==============================] - 57s 3s/step - loss: 0.0395 - accuracy: 0.4458
Epoch 2/100
9/9 [==============================] - 9s 1s/step - loss: 0.0222 - accuracy: 0.4203
Epoch 3/100
9/9 [==============================] - 9s 997ms/step - loss: 0.0204 - accuracy: 0.4058
Epoch 4/100
9/9 [==============================] - 9s 996ms/step - loss: 0.0195 - accuracy: 0.3816
Epoch 5/100
9/9 [==============================] - 9s 1s/step - loss: 0.0192 - accuracy: 0.4276
Epoch 6/100
9/9 [==============================] - 9s 972ms/step - loss: 0.0189 - accuracy: 0.3824
Epoch 7/100
9/9 [==============================] - 9s 978ms/step - loss: 0.0188 - accuracy: 0.4075
Epoch 8/100
9/9 [==============================] - 9s 988ms/step - loss: 0.0186 - accuracy: 0.4114
Epoch 9/100
9/9 [==============================] - 9s 978ms/step - loss: 0.0185 - accuracy: 0.4253
Epoch 10/100
9/9 [==============================] - 9s 1s/step - loss: 0.0184 - accuracy: 0.4259
Epoch 11/100
9/9 [==

In [13]:
# Step 1: Train the Autoencoder (you've already done this)
# autoencoder.fit(X_train, X_train, epochs=100, batch_size=32)

# Step 2: Extract the Encoder
# Define the encoder model
encoder_model = models.Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('conv2d_4').output)

# Print the encoder model summary
encoder_model.summary()


# Step 3: Create the Classifier
classifier_input = layers.Input(shape=(32, 32, 512))  # Set the correct input shape to match the output shape of the encoder
x = layers.Flatten()(classifier_input)
x = layers.Dense(64, activation='relu')(x)
classifier_output = layers.Dense(1, activation='sigmoid')(x)
classifier_model = models.Model(inputs=classifier_input, outputs=classifier_output)  # Use classifier_input as the input to the classifier model
classifier_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classifier_model.summary()



# Apply the encoder to get the lower-dimensional representations (encoded images)
encoded_images_training = encoder_model.predict(X_train_classify)
encoded_images_test = encoder_model.predict(X_test_classify)

# Train the classifier using the encoded images and corresponding labels
classifier_model.fit(encoded_images_training, y_train_classify, epochs=100, batch_size=32)

# Step 5: Evaluate the Classifier with a completely data
loss, accuracy = classifier_model.evaluate(encoded_images_test, y_test_classify)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")



Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 512, 512, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 512, 512, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 256, 256, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 128, 128, 64)     0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 128, 128)     7385

In [ ]:
# # Assuming X_test contains your test data
# loss = autoencoder.evaluate(X_test, X_test)

# print(f"Test Loss: {loss}")
# # Assuming X_test contains your test data
# reconstructions = autoencoder.predict(X_test)
# print(X_test)
# mse = np.mean(np.square(X_test - reconstructions))

# print(f"Mean Squared Error (MSE): {mse}")

# from keras import metrics

# loss, accuracy = autoencoder.evaluate(X_test, X_test)
# print(f"Test Loss: {loss}")
# print(f"Test Accuracy: {accuracy}")

4/4 [==============================] - 6s 2s/step - loss: 0.6211 - accuracy: 0.8090
Test Loss: [0.6210716962814331, 0.8089950680732727]
4/4 [==============================] - 1s 140ms/step
Mean Squared Error (MSE): 0.003011921187862754
4/4 [==============================] - 1s 223ms/step - loss: 0.6211 - accuracy: 0.8090
Test Loss: 0.6210716962814331
Test Accuracy: 0.8089950680732727


In [ ]:
model_classify_all.fit(X_train,y_train,validation_data=(X_test,y_test), epochs=100)

In [ ]:
# 이번엔 그냥 정상 비정상으로만 분류하는거 도전해보기
simple_df = df.copy()

import pandas as pd

# Replace non-zero values in the 'target' column with 1
simple_df.loc[simple_df['target'] != 0, 'target'] = 1

# Print the updated DataFrame
simple_df['target'].value_counts()

0    308
1     89
Name: target, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X1_train, X1_test, y1_train, y1_test = train_test_split(simple_df['image'], simple_df['target'], test_size=0.2, stratify=simple_df['target'])

0    246
1     71
Name: target, dtype: int64

In [ ]:
import numpy as np

X1_train = X1_train.to_numpy()
X1_test = X1_test.to_numpy()
y1_train = y1_train.to_numpy()
y1_test = y1_test.to_numpy()

print(type(X1_train))

X1_train = np.array(X1_train.tolist())
X1_test = np.array(X1_test.tolist())
y1_train = np.array(y1_train.tolist())
y1_test = np.array(y1_test.tolist())

print(type(X1_train))

X1_train = tf.convert_to_tensor(X1_train, dtype=tf.float32)
X1_test = tf.convert_to_tensor(X1_test, dtype=tf.float32)
y1_train = tf.convert_to_tensor(y1_train, dtype=tf.float32)
y1_test = tf.convert_to_tensor(y1_test, dtype=tf.float32)

print(type(X1_train))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [ ]:
# import tensorflow as tf
# from tensorflow.keras import layers, models, callbacks

# # Define the model
# model = models.Sequential()

# # Convolutional layers
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3), padding='same'))
# model.add(layers.MaxPooling2D((2, 2),padding='same'))
# model.add(layers.Dropout(0.2))  # Add Dropout with 20% rate

# model.add(layers.Conv2D(64, (3, 3), activation='relu' , padding= 'same'))
# model.add(layers.Dropout(0.2))  # Add Dropout with 20% rate

# model.add(layers.Conv2D(128, (3, 3), activation='relu'))
# model.add(layers.Dropout(0.2))  # Add Dropout with 20% rate

# model.add(layers.Conv2D(256, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Dropout(0.2))  # Add Dropout with 20% rate

# model.add(layers.Conv2D(512, (3, 3), activation='relu'))  # New convolutional layer
# model.add(layers.Dropout(0.2))  # Add Dropout with 20% rate

# # Flatten the feature maps
# model.add(layers.Flatten())

# # Dense layers
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dropout(0.5))  # Add Dropout with 50% rate
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dropout(0.5))  # Add Dropout with 50% rate
# model.add(layers.Dense(1, activation='sigmoid'))  # Sigmoid activation for binary classification

# # Compile the model
# model.compile(optimizer='adam',
#               loss='binary_crossentropy',
#               metrics=['accuracy'])

# # Print the model summary
# model.summary()

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

# Define the model
model = models.Sequential()

# Convolutional layers
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(layers.Dropout(0.2))  # Add Dropout with 20% rate

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2), padding= 'same'))
model.add(layers.Dropout(0.2))  # Add Dropout with 20% rate

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten the feature maps
model.add(layers.Flatten())

# Dense layers
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))  # Sigmoid activation for binary classification

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 254, 254, 32)      896       
                                                                 
 dropout_15 (Dropout)        (None, 254, 254, 32)      0         
                                                                 
 conv2d_21 (Conv2D)          (None, 252, 252, 64)      18496     
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 126, 126, 64)     0         
 g2D)                                                            
                                                                 
 dropout_16 (Dropout)        (None, 126, 126, 64)      0         
                                                                 
 conv2d_22 (Conv2D)          (None, 124, 124, 128)     73856     
                                                      

In [ ]:

# Train the model
history = model.fit(X1_train, y1_train,
                    validation_data=(X1_test, y1_test),
                    epochs=200,
                    )

# Find the best model based on validation accuracy
best_epoch = np.argmax(history.history['val_accuracy']) + 1
best_validation_accuracy = max(history.history['val_accuracy'])
print(f"Best model is at epoch {best_epoch} with validation accuracy of {best_validation_accuracy:.4f}")


Epoch 1/200
10/10 [==============================] - 15s 866ms/step - loss: 5.4371 - accuracy: 0.6246 - val_loss: 0.5347 - val_accuracy: 0.7750
Epoch 2/200
10/10 [==============================] - 3s 298ms/step - loss: 0.5478 - accuracy: 0.7760 - val_loss: 0.5344 - val_accuracy: 0.7750
Epoch 3/200
10/10 [==============================] - 3s 295ms/step - loss: 0.5554 - accuracy: 0.7760 - val_loss: 0.6071 - val_accuracy: 0.7750
Epoch 4/200
10/10 [==============================] - 3s 293ms/step - loss: 0.5286 - accuracy: 0.7760 - val_loss: 0.5559 - val_accuracy: 0.7750
Epoch 5/200
10/10 [==============================] - 3s 294ms/step - loss: 0.5548 - accuracy: 0.7760 - val_loss: 0.5233 - val_accuracy: 0.7750
Epoch 6/200
10/10 [==============================] - 3s 299ms/step - loss: 0.5636 - accuracy: 0.7760 - val_loss: 0.6502 - val_accuracy: 0.7750
Epoch 7/200
10/10 [==============================] - 3s 298ms/step - loss: 0.5604 - accuracy: 0.7760 - val_loss: 0.5218 - val_accuracy: 0.775

KeyboardInterrupt: ignored